<a href="https://colab.research.google.com/github/katduecker/nma_dl_bearmin/blob/main/bearmin_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# @title Install dependencies
!pip install livelossplot --quiet
!pip install Pillow --quiet
!pip install imageio-ffmpeg --quiet

!pip install git+https://github.com/NeuromatchAcademy/evaltools --quiet
from evaltools.airtable import AirtableForm

# generate airtable form
atform = AirtableForm('appn7VdPRseSoMXEG','W2D1_T1','https://portal.neuromatchacademy.org/api/redirect/to/9c55f6cb-cdf9-4429-ac1c-ec44fe64c303')

     |████████████████████████████████| 26.9 MB 90 kB/s 


In [12]:
# Imports
import time
import torch
import pathlib
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset

from tqdm.notebook import tqdm, trange
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# @title Set device (GPU or CPU). Execute `set_device()`
# especially if torch modules used.

# inform the user if the notebook uses GPU or CPU.

def set_device():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device


#set_device()

In [16]:
DEVICE = set_device()
DEVICE

'cpu'

In [18]:
# load data
x = np.load('/content/drive/My Drive/Colab Notebooks/NMA DL 2021/bear mins/cnn_ecog_tfr_loghouse_t.npz',allow_pickle=True)
logpowhouse_t = x['arr_0'] # access array that contains tfrs trial x frequency x time x channel

x = np.load('/content/drive/My Drive/Colab Notebooks/NMA DL 2021/bear mins/cnn_ecog_tfr_logface_t.npz',allow_pickle=True)
logpowface_t = x['arr_0']

logpowhouse_t.shape, logpowface_t.shape

del x

In [20]:
# one-hot code outputs
lface = np.hstack((np.ones((logpowface_t.shape[0],1)), np.zeros((logpowface_t.shape[0],1))))  # output face [1 0]
lhouse = np.hstack((np.zeros((logpowhouse_t.shape[0],1)), np.ones((logpowhouse_t.shape[0],1)))) # output house  [0 1]

y = np.vstack((lface,lhouse))  # concatenate vertically (150 faces, 150 houses)
         

In [24]:
data = np.vstack((logpowface_t,logpowhouse_t)) # concatenate tfrs vertically (same order as outputs y)

del logpowface_t, logpowhouse_t

datnorm = torch.nn.functional.normalize(torch.from_numpy(data), p=2.0, dim=1, eps=1e-12, out=None).to(DEVICE)

del data

# normalize: subtract mean

In [47]:
szmini = 10  # size minibatch
shufidx = np.random.permutation(datnorm.shape[0]) # randomly drawn indices

# split into train & test
idxtrain = shufidx[0:200]
idxtest = shufidx[200:datnorm.shape[0]]

# template
#for i in range(data.shape[0]/szmini):
  #curx = data[shufidx[i*szmini:i*szmini+szmini],:,:,:]
  #cury = y[shufidx[i*szmini:i*szmini+szmini]]

In [49]:
# convert to tensor

# training set
datatrain = torch.from_numpy(data[idxtrain,:,:,:])
labeltrain = torch.from_numpy(y[idxtrain,:])

# test set
datatest = torch.from_numpy(data[idxtest,:,:,:])
labeltest = torch.from_numpy(y[idxtest,:])



In [ ]:
# create cnn